In [1]:
import torch
from transformers import BertTokenizer, BertModel, BertConfig
import numpy as np

from xai_transformer import BertAttention
from xai_transformer import LNargsDetach, LNargsDetachNotMean
from utils import LayerNorm

🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py


W1119 05:32:34.511000 930 torch/utils/cpp_extension.py:118] No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
GroupViT models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version.Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.
TAPAS models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version. Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.


In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/tokenizer_config.json
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/tokenizer.json
loading file chat_template.jinja from cache at None


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [3]:
text = "This movie was absolutely wonderful, I loved it."

In [4]:
encoding = tokenizer(text, return_tensors='pt')
input_ids = encoding["input_ids"]
print("Token IDs:", input_ids)

Token IDs: tensor([[ 101, 2023, 3185, 2001, 7078, 6919, 1010, 1045, 3866, 2009, 1012,  102]])


In [5]:
bert_config = BertConfig.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/config.json
Model config BertConfig {
  "architec

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594/model.safetensors
A pretrained model of type `BertModel` contains parameters that have been renamed internally (a few are listed below but more are present in the model):
* `bert.encoder.layer.9.output.LayerNorm.beta` -> `encoder.layer.9.output.LayerNorm.bias`
* `bert.encoder.layer.9.output.LayerNorm.gamma` -> `encoder.layer.9.output.LayerNorm.weight`
If you are using a model from the Hub, consider submitting a PR to adjust these weights and help future users.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.beta', 'cls.predictions.transform.LayerNorm.gamma', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected 

In [6]:
embeddings = bert_model.embeddings
embeddings

BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [20]:
## we have replaced the LayerNorm by our custom no gradient implementation
from utils import LayerNorm
embeddings.LayerNorm = LayerNorm(bert_config.hidden_size,
                                 eps = bert_config.layer_norm_eps,
                                 args = LNargsDetach())

In [21]:
embeddings

BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNormImpl()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [30]:
class SimpleConfig:
    hidden_size = bert_config.hidden_size
    all_head_size = bert_config.hidden_size
    num_attention_heads = bert_config.num_attention_heads
    attention_head_size = bert_config.hidden_size // bert_config.num_attention_heads
    n_blocks = 4
    n_classes = 2
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    train_mode = False

    # complete detached run
    detach_layernorm = True
    detach_mean = True
    detach_kq = True

    layer_norm_eps = 1e-5

    #
    lnv = 'nowb'
    sigma = None
    adanorm_scale = 1.
    nowb_scale = None
    layer_norm_eps = 1e-5

In [31]:
config = SimpleConfig()

In [32]:
model = BertAttention(config, embeddings).to(config.device)

print("\nModel initialized. Running baseline explanation...\n")

Detach LayerNorm Mean+Norm
Detach K-Q-softmax branch
Detach LayerNorm Mean+Norm
Detach K-Q-softmax branch
Detach LayerNorm Mean+Norm
Detach K-Q-softmax branch
Detach LayerNorm Mean+Norm
Detach K-Q-softmax branch
Detach LayerNorm Mean+Norm

Model initialized. Running baseline explanation...



In [33]:
target_class = 1

result = model.forward_and_explain(
    input_ids=input_ids,
    cl=target_class,
    gammas = None
)

In [34]:
result

{'loss': None,
 'logits': tensor([[0.0315, 0.3571]], grad_fn=<AddmmBackward0>),
 'R': array([[ 0.12551929, -0.00686899,  0.02330785,  0.04297955,  0.03065391,
          0.01630355, -0.00105841,  0.0396429 ,  0.05181998,  0.02050681,
         -0.00354076, -0.03387561]], dtype=float32)}

In [35]:
relevance = result["R"] # shape: [1, seq_len]
logits = result["logits"]

In [36]:
print("Relevance shape:", relevance.shape)
print("Logits:", logits)
print("Relevance:", relevance)

Relevance shape: (1, 12)
Logits: tensor([[0.0315, 0.3571]], grad_fn=<AddmmBackward0>)
Relevance: [[ 0.12551929 -0.00686899  0.02330785  0.04297955  0.03065391  0.01630355
  -0.00105841  0.0396429   0.05181998  0.02050681 -0.00354076 -0.03387561]]


In [42]:
f_x = logits[0, target_class].item()
sum_relevance = float(np.sum(relevance))

print("Conservation Check\n")
print("Model output f(x): \n", f_x)
print()

print("Sum of relevance R(x_i):\n", sum_relevance)
print()

print("Difference:\n", abs(f_x - sum_relevance))
print()

Conservation Check

Model output f(x): 
 0.3570597767829895

Sum of relevance R(x_i):
 0.3053900897502899

Difference:
 0.051669687032699585

